### 타이틀 끝에 붙는 ~종합, ~종합n보 일괄 삭제

In [6]:
def preprocess_article_title(title):
    # Check if the title ends with "~" and a suffix
    if '종합' in title[-4:]:
        last_space_index = title.rfind('종합')
        # print("Index of the last space:", title[:last_space_index])
        cleaned_title = title[:last_space_index]

        return cleaned_title
    else:
        return title

# Example usage:
article_title = "중국 무역전쟁 속 최대 이벤트 양회 개막종합2보"
cleaned_title = preprocess_article_title(article_title)

print("Original Title:", article_title)
print("Cleaned Title:", cleaned_title)

Original Title: 중국 무역전쟁 속 최대 이벤트 양회 개막종합2보
Cleaned Title: 중국 무역전쟁 속 최대 이벤트 양회 개막


### 한자 -> 한글 변환

In [ ]:
import hanja, re
from hanja import hangul

is_hanja = []
hanja_to_korean = []
for idx, line in df.iterrows():
    if re.match('[/一-龥/]', line['text']):
        print(line['text'])
        is_hanja.append(1)
        hanja_to_korean.append(hanja.translate(line['text'], 'substitution'))
    else:
        is_hanja.append(0)
        hanja_to_korean.append('')

df['include_hanja'] = is_hanja
df['hanja_translate'] = hanja_to_korean

### 보이지 않는 유니코드 문자, [신간], [지역] 등 불용어 및 깨진 문자 복구

In [1]:
import re
def clean_sentence(sentence):
    # Define a pattern to match both double and single quotes
    quote_pattern = r"[\"'‘’“”]"

    # Use the sub() function to replace matched quotes with an empty string
    result = re.sub(quote_pattern, '', sentence)

    # Define a pattern to match bracket pairs with 2 or fewer letters inside
    short_word_pattern = r"\[[a-zA-Z가-힣]{,2}\]"

    # Use the sub() function to replace matched bracket pairs with an empty string
    result = re.sub(short_word_pattern, '', result)

    # Define a pattern to match and remove invisible Unicode characters
    invisible_char_pattern = r"[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F\u200B\u200C\u2028\u2029\u00A0]"
    # \u00A0 -> invisible unicode character
    
    # Use the sub() function to replace matched invisible characters with an empty string
    result = re.sub(invisible_char_pattern, '', result, flags=re.UNICODE)
    result = result.replace('&lt;', '<').replace('&gt;', '>')

    return result
# ==========================================================================


In [4]:
input_sentence = "This is a 'sample' sentence with \"quotes\" and [go].\u200B\u200C"
output_sentence = clean_sentence(input_sentence)
print(output_sentence)

This is a sample sentence with quotes and .


In [5]:
input_sentence = "[신간] 황현택 두 번째 시집 ‘뜬봉샘’"
output_sentence = clean_sentence(input_sentence)
print(output_sentence)

 황현택 두 번째 시집 뜬봉샘


### 모델이 얼마나 정답에 가깝게 추론하는지, 오답은 어떤 유형의 오답을 기록하는지 탐색

In [ ]:
out_train = pd.read_csv('../code/output_train.csv')
train = pd.read_csv('../data/train.csv')

In [ ]:
cnt = 0
diff = pd.concat([train['target'], out_train['target'], train['text']], axis=1)
diff.columns = ['ans', 'pred', 'text']

diff

In [ ]:
for idx, row in diff.iterrows():
    if row['ans'] == row['pred']:
        continue
    else:
        print(idx, row['and'], row['pred'], row['text'])
        cnt += 1
print(cnt)